# Overview

We consider Chapter $5$ of the thesis. In particular we implement the Algorithm of Section $5.1$ and generate $\tilde{P}_{n}$ for $1 \le n\le 6$  from $\tilde{P}_{n-1}$. We then give a decomposition of $p_n:=|\tilde{P}_n|$ in terms of $|\tilde{P}_{n-1}|$.

Recall that,

$$P_n=\bigsqcup_{g \in P_{n-1}} \rho^{-1}(g)= \bigsqcup_{g \in \tilde{P}_{n}}\{g+\epsilon_{I} | I \in \mathbb{E}(g)\}.$$

The **procedure** to recursively obtain $\tilde{P}_{n}$ from $\tilde{P}_{n-1}$ is as follows:
 
 1) for each  $g \in \tilde{P}_{n-1}$ obtiain the set $\mathbb{E}(g)$ (using the Algorithm of Section $5.1$ )
 
     1.1) for each  $g \in \tilde{P}_{n-1}$ we record $|\mathbb{E}(g)|$ (which allows one to obtain the sequences $e_{n}$ and
     $d_{n}$ which allow for $p_{n+1}=e_n \cdot d_n$)
     
 2) for each set $\mathbb{E}(g)$ obtain the set $\{g+\epsilon_{I} | I \in \mathbb{E}(g)\}$
 
 3) take the union as above to obtain $\tilde{P}_{n}$.

## Results

We obtain the sequence of $p_n$ for $1 \le n\le 6$.

  $$1,2,10,154,10334,5399325.$$

We also have that,

$$p_6=5399325=114*125+121*65+126*237+138*112+158*248+162*247+165*132+169*117+173*132+178*122+182*106+183*132+187*24+192*134+194*123+208*144+209*22+216*234+217*126+218*61+221*125+227*22+229*307+230*250+232*120+233*119+236*122+241*30+242*102+247*61+248*53+250*114+251*59+260*184+268*106+269*64+271*186+273*118+274*56+278*52+279*41+281*136+284*112+289*67+293*133+294*37+300*133+305*67+313*123+324*123+325*132+326*101+329*181+338*120+343*59+345*113+356*19+381*121+390*44+397*107+419*20+420*46+421*23+422*39+429*64+433*35+436*128+437*23+455*64+477*53+496*48+502*16+509*38+518*12+520*107+524*20+531*36+538*76+539*62+553*36+555*66+566*144+578*61+605*62+607*66+613*11+640*28+659*30+759*48+765*21+781*108+802*119+814*42+881*42+938*8+974*135+977*45+983*135+1002*45+1026*126+1029*63+1033*63+1068*96+1121*108+1145*22+1178*39+1189*39+1209*44+1212*11+1224*66+1257*44+1312*11+1322*32+1377*36+1409*16+1421*10+1450*54+1511*20+1666*21+1753*24+2110*42+2193*95+2285*4+2290*54+3544*12+3673*42+3820*48+3983*18+6280*1+6474*6+6702*14+6960*16+7250*9+7580*4$$


where the decomposition of the remaining $p_{n}$ are stated in Section $5.1$.

In "4) Example $\mathbb{E}(g)$ for $g \in  \tilde{P}_{3}$" we provide a method to obtain $\mathbb{E}(g)$ for any $g \in \tilde{P}_{n}$.

## Structure

The main content of this file is in the section #$\tilde{P}_3,\tilde{P}_4,\tilde{P}_5$ and #Decomposition of $|\tilde{P}_n|$ for $n\le 5$.

In  #$\tilde{P}_3,\tilde{P}_4,\tilde{P}_5$ we have the implementation of the algorithm in Section $5.1$ in "Functions", and dataframes of $\tilde{P}_3,\tilde{P}_4,\tilde{P}_5$ in "Cases".


In section #$\tilde{P}_6$ we apply the same procedure as in the previous cases, except we break the compution into parts due to its computational time.

## Functions used throughout

In [1]:
import itertools
import pandas as pd
import numpy as np
import pickle
import time
import ast
import inspect

# Table of contents

1. [$\tilde{P}_3,\tilde{P}_4,\tilde{P}_5$](#s1)
    1. [Functions](#s11)
        1. [General notation](s111)
        2. [To obtain $\mathbb{E}(g)$ for $g \in \tilde{P}_{n-1}$](s112)
        3. [Obtaining $\tilde{P}_{n+1}$ from $\mathbb{E}(g)$ from $g \in \tilde{P}_{n-1}$](s113)
        4. [Analysis of $\tilde{P}_{n}$](s114)
    2. [Cases](#s12)
        1. [$\tilde{P}_{3}$ by $\tilde{P}_{2}$](s121)
        2. [$\tilde{P}_{4}$ by $\tilde{P}_{3}$](s122)
        3. [$\tilde{P}_{5}$ by $\tilde{P}_{4}$](s123)
    
2. [Decomposition of $|\tilde{P}_n|$ for $n \le 5$](#s2)
    1. [Functions](#s21)
    2. [Cases](#s22)
        1. [Decomposition of $|\tilde{P}_3|$](#s221)
        2. [Decomposition of $|\tilde{P}_4|$](#s222)
        3. [Decomposition of $|\tilde{P}_5|$](#s223)
        
3. [Which $g \in \tilde{P}_{n-1}$ attain max and min $|\mathbb{E}(g)|$](#s3)
    1. [$U_3$](#s31)
    2. [$L_3$](#s32)
    

4. [$\tilde{P}_6$ and decomposition of $|\tilde{P}_6|$ (Debt)](#s4)

# $\tilde{P}_3,\tilde{P}_4,\tilde{P}_5$ <a name="s1"></a>

## Functions <a name="s11"></a>

Before recursively constructing $\tilde{P}_{n}$ we first need to implement the notation and **procedure**.

### General notation:<a name="s111"></a>

The following functions allow us to store $g \in \tilde{P}_n$ as a dictionary (e.g. of the form {'1': 0, '2': 0, '12': 0}) in particular the keys as a string representation of $I \in \mathcal{P}_{n}^{+}$ and to change back to sets notation when needed.

In [2]:
def set_to_string(s):
    """
    Objective: Convert a set of integers to a sorted string representation.
    
    Input:
        - s: A set of integers.
        
    Returns:
        - result: A string containing the sorted integers from the input set.
    """
    
    
    s = list(s)
    s.sort()
    # Convert the list of numbers to a list of strings
    my_list=s
    my_list = [str(i) for i in my_list]
    # Use the join() function to convert the list of strings to a single string
    result = ''.join(my_list)
    return result

def string_to_set(s):
    """
    Objective: Convert a string of digits to a set of integers.
    
    Input:
        - s: A string of digits.
        
    Returns:
        - my_set: A set containing the integers parsed from the input string.
    """
    my_list = list(s)
    # Convert the list of strings to a list of integers
    my_list = list(map(int,my_list))
    # Convert the list to a set
    my_set = set(my_list)
    return my_set

The following functions allow us to state $\mathcal{P}_{n}^{+}=\mathcal{P}_{n} \setminus \emptyset$ and $\mathcal{P}(\mathcal{P}_{n}^{+})$ (where $\mathcal{P}_{n}$ is the powerset of $\{1,\dots ,n\}$).

In [3]:
def get_powerset_str(n):
    """
    Objective: Generate the power set of integers from 1 to n and represent it as a list of sorted strings.
    
    Input:
        - n: An integer representing the upper bound of the set of integers (1 to n).
        
    Returns:
        - power_set_str: A list of sorted strings representing the power set of integers from 1 to n.
    """
    s=set(range(1,n+1))
    power_set = [set(x)  for r in range(len(s) + 1) for x in itertools.combinations(s, r)]
    power_set_str=[set_to_string(x) for x in power_set if len(x)>0]

    return power_set_str

def get_n_np1_powersets(n):
    """
    Objective: Generate the power sets of integers from 1 to n and 1 to (n+1), representing them as lists of sorted strings.
    
    Input:
        - n: An integer representing the upper bound of the sets of integers (1 to n and 1 to n+1).
        
    Returns:
        - power_set_str: A list of sorted strings representing the power set of integers from 1 to n.
        - power_set_str_np1: A list of sorted strings representing the power set of integers from 1 to n+1.
    """
    s=set(range(1,n+1))
    power_set = [set(x)  for r in range(len(s) + 1) for x in itertools.combinations(s, r)]
    power_set_str=[set_to_string(x) for x in power_set if len(x)>0]

    np1=n+1
    sp1=set(range(1,n+2))
    power_set_np1 = [set(x)  for r in range(np1+1 + 1) for x in itertools.combinations(sp1, r)]
    power_set_str_np1=[set_to_string(x) for x in power_set_np1 if len(x)>0]

    return power_set_str,power_set_str_np1

As sets are mutable we use the following function to put them into a starndard form and make them immutable.

In [4]:
def sorted_frozenset(s):
    """
    Objective: Sort the elements of a set and return them as a frozenset.
    
    Input:
        - s: A set of elements.
        
    Returns:
        - r: A frozenset containing the sorted elements from the input set.
    """
    t=sorted(s)
    r=frozenset(t)
    return r

###  To obtain $\mathbb{E}(g)$ for $g \in \tilde{P}_{n-1}$ <a name="s112"></a>

In this section we provide a method to return for a given  $g \in \tilde{P}_{n-1}$ the set $\mathbb{E}(g)$ (using the Algorithm of Section $5.1$ ) we do this through the function get_Eg(). We break this section into the following parts:

1) Obtain a method to determine for $A,B \in \mathcal{P}_{n}^{+}$ such that $A \subseteq B$ to state when $A \subseteq B$ is $f$-minimal or $f$-maximal. 

2) Obtain a method to obtain the closure of any $I \subseteq \in \mathcal{P}_{n}^{+}$.

3) Implement the Algorithim of Section $5.1$ to obtain $\mathbb{E}(g)$ for any $g$.

4) Consider an example of $\mathbb{E}(g)$ for $g \in  \tilde{P}_{3}$.


#### 1) $f$-minimality or $f$-maximality

Let $f \in \tilde{P}_{n}$ and $A,B \in \mathcal{P}_{n}^{+}$ such that $A \subseteq B$.  The following functions determine if $A \subseteq B$ is $f$-minimal or $f$-maximal.

In [5]:
def fmin(A,B,f_dict):
    """
    Objective: Determine if A is f-minimal with respect to B using a given dictionary of values.

    Input:
        - A: A string representing a set from the power set P_n.
        - B: A string representing another set from the power set P_n where A is a subset of B.
        - f_dict: A dictionary mapping set strings to corresponding values.

    Returns:
        - True if A is f-minimal with respect to B, False otherwise.
    """
    
    #A included in B
    # global f_dict
    
    valA=f_dict[A]
    valB=f_dict[B]
    compBA=set_to_string(string_to_set(B)-string_to_set(A)) # B minus A
    if len(compBA)==0: # to avoide when taking the complement gives empty set
        return False
    valBA=f_dict[compBA]
    if valB==valA+valBA:
        return True
    else:
        return False
    
def fmax(A,B,f_dict):
    """
    Objective: Determine if A is f-maximal with respect to B using a given dictionary of values.

    Input:
        - A: A string representing a set from the power set P_n.
        - B: A string representing another set from the power set P_n where A is a subset of B.
        - f_dict: A dictionary mapping set strings to corresponding values.

    Returns:
        - True if A is f-maximal with respect to B, False otherwise.
    """
    #A included in B
    # global f_dict
    
    valA=f_dict[A]
    valB=f_dict[B]
    compBA=set_to_string(string_to_set(B)-string_to_set(A)) # B minus A
    
    #A has to be a subset of B first
    if len(compBA)==0: # to avoide when taking the complement gives empty set
        return False
    valBA=f_dict[compBA]
    if valB==valA+valBA+1:
        return True
    else:
        return False

#### 2) Implementation of closure of and $I \subseteq \mathcal{P}_{n}^{+}$



Recall Definition $5.1.4$. That the closure of some $I \subseteq \mathcal{P}_{n}^{+}$ denoted as, $\overline{I}$, with respect to $g$ is be the largest set in the following sequence of subsets of $\mathcal{P}_{n}^+$,

$$I=I_0 \subseteq  \cdots \subseteq I_k $$

where $I_{i}:=B_{i-1}\cup I_{i-1}$ (we call this the $i$-partial closure of $I$) such that 

$$    B_{i-1}=  \left\{y \in \mathcal{P}_{n}^{+} \mid \forall x \in I_{i-1} \text{ such that } y \supsetneq x \text{ is $g$-minimal} \text{ or }
y \subsetneq x  \text{ is $g$-maximal} \right\}.$$ 




The next function takes $I:=I_0 \in \mathcal{P}_{n}^{+}$ and returns $I_{1}$ as defined above.


In [6]:
def next_j(ji,f_dict):
    """
    Objective: Get the $i+1$-partial closure of $I$ of ji with respect to f_dict. 
    
    By iterating through elements in ji and checks conditions involving fmin and fmax to determine which elements should be included in the new set jip1.
    
    Input:
    #ji is the ith recursion set on constructing the closure. 
    f_dict: the function ususal denoted as g
    
    Returns: closure of ji.
    """
    # print("ji",ji)
    # global f
    
    my_list=[]    
    
    
    for x in power_set_str:
        for y in ji:
            if fmin(y,x,f_dict) and string_to_set(y).issubset(string_to_set(x)):
                # print("y is",y)
                my_list.append(x)
                # print(f"{x} containing {y} f min ")
    for x in power_set_str:
        for y in ji:
            if fmax(x,y,f_dict) and string_to_set(x).issubset(string_to_set(y)):
                my_list.append(x)
                
    
    jip1=set(my_list).union(ji)
    
    return jip1    

The following function allows us to go from $i$-partial closure of $I$ to the $i+1$-partial closure of $I$ and continue this until we end at the closure of $I$ wrt $g$.

In [7]:
def rec_j(T,S,f_dict):
    
    
    """
    Objective: Given the $i$-partial closure of $S:=Ii$ to the $i+1$-partial closure of $T:=Ii+1$
    Input:
    T: (T=Ii)
    S : S=Ii-1)
    
    f_dict : function
    
    Returns: The closure of S.
    """
    # T=Ji and S=Ji-1, |T|\ge |S|.
    
    
    """
    We ask if the set T=S. If so them T is closed, if not we are not finished taking the closure.
    """
    C=string_to_set(T)-string_to_set(S) #T \S # New elements
    C={str(x) for x in C}    
    # print("C",C,f"S term {S}",f"T term {T}")
    
    #We are done
    if len(C)==0: #Check if closed set 
        return T # return closed set
    
    #Not done yet.
    if len(C)>0:
        """We then take the next partial-closure of T wrt Definion 5.1.4"""
        U=next_j(T,f_dict) #next_j(T)
        
        #We then repeat this function.
        Z=rec_j(U,T,f_dict)
    # print(T)
    return Z

Finally the last function combines next_j() and rec_j() to give the closure of $I$ with respect to $g$.

In [8]:
def get_barj(J,f_dict):
    """
    Objective: Gets the closure of J \subseteq \mathcal{P}_{n}^{+} with respect to f in \tilde{P}_n.
    Input:
    J: is a element of the \mathcal{P}_{n}^{+}
    f_dict in \tilde{P}_n.
    
    Returns:closure of J
    """
    
    j0=J
    j1=next_j(j0,f_dict)
    T=rec_j(j1,j0,f_dict)
    return T

Note that we can use get_barj() to immediatley obtain all primitve closed sets.

In [11]:
# PC=[(x,get_barj({x},g)) for x in power_set_str]
# dict_PC=dict(PC)
# set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]

#### 3) Algorithim from Section $5.1$.

Algorithm: Obtain $\mathbb{E}(g)$

**Input:** $g \in \tilde{P}_{n}$

**Output:** $\mathbb{E}(g)$

1. **Initialization:** Set $\mathbb{E}(g) = \{\emptyset\}$

2. Define a recursive function: Recursion($\overline{I}, y$)

    1. Let $\overline{J} := \overline{I} \cup \overline{\{y\}}$
    2. Add $\overline{J}$ to $\mathbb{E}(g)$
    3. For each $w \in \mathcal{P}_{n}^{+} \setminus \overline{J}$, do the following:
        1. Call Recursion($\overline{J}, w$)

3. **End Function**

4. For each $x \in \mathcal{P}_{n}^{+}$, do the following:
    1. Let $I := \{x\}$
    2. Add $\overline{I}$ to $\mathbb{E}(g)$
    3. For each $y \in \mathcal{P}_{n}^{+} \setminus \overline{I}$, do the following:
        1. Call Recursion($\overline{I}, y$)


The following function is the recusive part of above algorithm, in particular from step 4.C onwards. We do this separate the recursive part from the outer for loop for simplicity, as primitive closed sets are easy to obtain and so is $\mathcal{P}_{n}^{+}$ and $\emptyset$ which are always in $\mathbb{E}(g)$.

To enhance efficiency and reduce computation time of step 4.C. (so that we do not need to iterate through the powerset completely each time) we've introduced:

- **Memory:** We utilize the "Memory" variable as a memoization technique to store previous results, thus preventing redundant calculations.

These stored results are referred to as "indicators." Here's how it works:

- Before initiating a new recursion with a specific set of indicators (which track the current state), we check if these indicators are already present in Memory. If they are, we skip the associated calculations, saving processing time.
- If a set of indicators is not found in Memory, it signifies that our algorithm hasn't encountered this specific combination previously. Consequently, we proceed with the recursion and store this set of indicators in "Memory."

In [9]:
def Recur(Ibar,indicators,Eg,Memory,dict_PC):
    """
    Objective: Recursively generate Eg while preventing repeated calculations using Memory.

    Input:
    - Ibar (set): The set closure of I for some I \subseteq \mathcal{P}{n}^{+}
    
    - indicators (tuple): A tuple of elements of \mathcal{P}{n}^{+} indicators 
    indicators store recursive paths we have taking that is 
    e.g when we take y in the complement of the closure of x. 
    So for the algorithim stated above example indicators would be (x,y) (x,y,w)
    We write indicators in a standard ordered form to help memoization process

    - Memory (set): A set used for memoization to prevent repeated calculations storing tuples of indicators

    - Eg (set): Begins as the set of primitive closed sets and ends at the full set Eg
    
    - dict_PC (dict): A dictionary recording the element of p_n^{+} and the primitive closed sets.

    Returns:
    - Eg (set): The updated set of generated primitive closed sets.
    - Memory (set): The updated set of indicators used for memoization.
    """
    # global dict_PC
        
    # To save recalulating we uses this series of checks, if fails checks end recursion turn.
    
    if indicators in Memory:# Memortisation for preventing repeated calculations.
        # print("Fails Memory check")
        # print(f"\n At this failure we have \n Eg: {len(Eg)} \n Memory: {len(Memory)} \n")
        return Eg,Memory
    
    #Steps 2A and 2B of algo
    
    y=indicators[-1]
    # print(f"term taken for primitive closed set we are unioning {y}")
    P_y=dict_PC[y] # the primitive closed set wrt y
    Jbar= Ibar.union(P_y)
    
    # End if alread have Jbar
    
    if len(Jbar)==len(power_set_str): # We always have the power set in Eg
        # print(f"Fails as {Jbar} == poweset")
        # print(f"\n At this failure we have \n Eg: {len(Eg)} \n Memory: {len(Memory)} \n")
        return Eg,Memory
              
    if Jbar in Eg:
        # print(f"Fails as {Jbar} in Eg")
        # print(f"\n At this failure we have \n Eg: {len(Eg)} \n Memory: {len(Memory)} \n")
        return Eg,Memory
    
    #Repeat recursion step
    else:
        # print("Passes Checks \n")
        #Add the new data to memory
        
        set_indicators=set([sorted_frozenset(tuple(indicators)),])
        Memory=Memory.union(set_indicators)
        
        Jbar_union=set([sorted_frozenset(Jbar),])#Correct format for union
        #Add new data to Eg
        Eg=Eg.union(Jbar_union) # will ge an issue with where defines and globals.

        #Move onto the next recursion.
        power_complment=set(power_set_str)-Jbar
        for w in power_complment: # pick one in the complement to to continus exhaustive method
            new_indicators=indicators+(w,) #want to add it at end
            Eg,Memory=Recur(Jbar,new_indicators,Eg,Memory,dict_PC)
            # print(f"Done with {new_indicators} \n")
                
    return Eg,Memory # will also be an issue.


We now state the full algorithm. Note we obtain the primitive closed sets, $\mathcal{P}_{n}^{+}$ and $\emptyset$ separately.

In [10]:
def get_Eg(dict_PC,power_set_str,set_PC):
    """
    Objective: Compute the set of all closed sets, Eg, using a given dictionary of primitive closed sets (dict_PC),
               the power set of elements (power_set_str), and a set of primitive closed sets (set_PC).
               
               We do this by building up Eg using Recur()
               
               We deal with the empty set separatley.

    Input:
    - dict_PC (dict): A dictionary where keys are elements of the power set (as strings) 
    and values are primitive closed sets.
    - power_set_str (list or set): The power set of elements represented as a list or set.
    - set_PC (set): A set of primitive closed sets.

    Returns:
    - Eg (set): The set of all closed sets.
    """
    
    #We start with what we know which are the primitve closed sets.
    Eg={frozenset(power_set_str)}.union(set_PC) 
    

    Memory={sorted_frozenset(("0"))}    # contains indicators which are tuples of x \in powerset for primitives sets.
    
    for x in power_set_str: #(Step 1 of Algorthim)
        Ibar=dict_PC[x] #primitive closed set #(Step 2 of Algorthim)
        power_complment=set(power_set_str)-Ibar # \mathcal{P}_{n}^{+} \setminus \overline{I}
        for y in power_complment: #exhaustively getting all closed sets. #(Step 4 of Algorthim)
            
            #We introduce indicators which records that we are taking y in the complement of the closure of x
            indicators=(x,y)
            set_indicators=set([sorted_frozenset(tuple(indicators)),])
            
            Memory=Memory.union(set_indicators)            
            Eg,Memory=Recur(Ibar,indicators,Eg,Memory,dict_PC)
            
    return Eg

#### 4) Example $\mathbb{E}(g)$ for $g \in  \tilde{P}_{3}$ <a name="s32"></a>

Here we obtain the example in Section $5.1$ (which can be done by hand). This section is here to provide a check that the code written so far agrees with the theory.

Let $g \in \tilde{P}_3$,

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 0}

We now obtain all the primitive closed sets of $\mathbb{E}(g)$ from a single function input.

In [11]:
# df=Pn for some n
def get_primitive_for_single_g(df,numb):
    """
    Objective:
    Input:
    Returns: 
    
    set_PC: set of primitive closed set
    dict_PC: dictionary with keys as terms of powerset^{+} with value as the primitive closed assoicated.
    """
    #Inputs: df=Pn, numb used to get a specific function from df
    
    # Pick function in Pn
    g=df.iloc[numb].to_dict()

    #get primitive closed sets
    PC=[(x,get_barj({x},g)) for x in power_set_str]
    dict_PC=dict(PC)
    set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]


    return dict_PC,set_PC

In [12]:
#Initalisation: get P3
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])
n=2
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P3=get_Pnp1(P2,2)
df=P3
print(P3)
#    1  2  3  12  13  23  123
# 0  0  0  0   0   1   0    1
# 1  0  0  0   0   1   1    1
# 2  0  0  0   0   0   0    1
# 3  0  0  0   0   0   1    1
# 4  0  0  0   0   0   0    0
# 5  0  0  0   1   1   0    1
# 6  0  0  0   1   1   1    2
# 7  0  0  0   1   0   1    1
# 8  0  0  0   1   1   1    1
# 9  0  0  0   1   0   0    1

NameError: name 'get_Pnp1' is not defined

In [45]:
# pick a function
numb=9

#Get the powerset for P3
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)

In [46]:
print(P3.iloc[numb].to_dict(),"\n")
# {'1': 0, '2': 0, '3': 0, '12': 1, '13': 0, '23': 0, '123': 1} 

{'1': 0, '2': 0, '3': 0, '12': 1, '13': 0, '23': 0, '123': 1} 



In [47]:
#get primitive closed data.
dict_PC,set_PC=get_primitive_for_single_g(df,numb)

In [49]:
print("\n Primitive closed sets")
dict_PC

# {'1': {'1', '13'},
#  '2': {'2', '23'},
#  '3': {'1', '123', '13', '2', '23', '3'},
#  '12': {'1', '12', '123', '13', '2', '23'},
#  '13': {'13'},
#  '23': {'23'},
#  '123': {'1', '123', '13', '2', '23'}}

{'1': {'1', '13'},
 '2': {'2', '23'},
 '3': {'1', '123', '13', '2', '23', '3'},
 '12': {'1', '12', '123', '13', '2', '23'},
 '13': {'13'},
 '23': {'23'},
 '123': {'1', '123', '13', '2', '23'}}

We now obtain the set $\mathbb{E}(g)$.

In [52]:
Eg=get_Eg(dict_PC,power_set_str,set_PC)
# Eg.append("empty")
Eg
# {frozenset(),
#  frozenset({'13'}),
#  frozenset({'1', '13'}),
#  frozenset({'23'}),
#  frozenset({'1', '13', '23'}),
#  frozenset({'2', '23'}),
#  frozenset({'13', '23'}),
#  frozenset({'13', '2', '23'}),
#  frozenset({'1', '13', '2', '23'}),
#  frozenset({'1', '123', '13', '2', '23'}),
#  frozenset({'1', '12', '123', '13', '2', '23'}),
#  frozenset({'1', '123', '13', '2', '23', '3'}),
#  frozenset({'1', '12', '123', '13', '2', '23', '3'})}

{frozenset(),
 frozenset({'13'}),
 frozenset({'1', '13'}),
 frozenset({'23'}),
 frozenset({'1', '13', '23'}),
 frozenset({'2', '23'}),
 frozenset({'13', '23'}),
 frozenset({'13', '2', '23'}),
 frozenset({'1', '13', '2', '23'}),
 frozenset({'1', '123', '13', '2', '23'}),
 frozenset({'1', '12', '123', '13', '2', '23'}),
 frozenset({'1', '123', '13', '2', '23', '3'}),
 frozenset({'1', '12', '123', '13', '2', '23', '3'})}

### Obtaining $\tilde{P}_{n+1}$ from $\mathbb{E}(g)$ from $g \in \tilde{P}_{n-1}$ <a name="s113"></a>

In this section  for  $g \in \tilde{P}_{n-1}$ and $\mathbb{E}(g)$ we obtain the set $\{f=g+\epsilon_{I} | I \in \mathbb{E}(g)\}$. We then do this for all $g \in \tilde{P}_{n-1}$ and take the union in order to state $\tilde{P}_{n}$. The main function of this section is get_Pnp1(), which takes $\tilde{P}_{n-1}$ and returns $\tilde{P}_{n}$.


Let $f=g+\epsilon_{I}$ for some $I \in \mathbb{E}(g)$then $f(A \cup \{n+1\})= g(A)+\epsilon_{I}(A)$ for $A \in \mathcal{P}_{n-1}^{+}$. The following function allow us to obtain $A \cup \{n+1\} \in \mathcal{P}_{n}^{+}$.

In [13]:
def add_n_p_1_to_string(string,n):
    """
    Objective: Adds n+1 to the input string.

    Input:
    - string (str): The input string to which n+1 will be added.
    - n (int): The number to be added to the string.

    Returns:
    - np1string (str): A new string formed by concatenating the input string and (n+1).
    """
    #Adds n+1 to strings subsets
    np1string=string+(f"{n+1}")
    return np1string

Let $I \in \mathbb{E}(g)$ but $I\ne \emptyset$ (we consider $I= \emptyset$ separately) the following function returns the set of all $f=g+\epsilon_{I}$.

In [14]:
def get_extension_of_f(f_dict,extender,power_set_str,n):
    """
    Objective:
    Input:
    Returns:
    """
    # Need to attatch n+1 to strings to build extension of function by epsilon.

    # Build the extension of f by epsilon.
    
    # Add the (key,values) of f_dict
    builder_ext_0=[(k, f_dict[k]) for k in f_dict]

    #We add 1 to the function value of f_dict for x in extender
    builder_ext_p1=[(add_n_p_1_to_string(x,n),f_dict[x]+1) for x in list(extender)]

    #We add 0 to the function value of f_dict for x NOT in extender
    compl_extender=set(power_set_str) - set(extender)
    builder_ext_p2=[(add_n_p_1_to_string(x,n),f_dict[x]+0) for x in list(compl_extender)]
    
    # We include f({n+1})=0
    builder_ext_np1=[(add_n_p_1_to_string("",n),int(0)),]


    #Compile together.
    extension_f_dict={**dict(builder_ext_0),**dict(builder_ext_p1),**dict(builder_ext_p2),**dict(builder_ext_np1)}
    
    #Example
    # get_extension_of_f(frozenset({'13', '23'}))

    return extension_f_dict

Let $I= \emptyset \in \mathbb{E}(g)$ the following function returns $f=g+\epsilon_{I}$.

In [15]:
def empty_case_get_extension_of_f(f_dict,extender,power_set_str,n):
    """
    Objective:
    Input:
    Returns:
    """
    # Need to attatch n+1 to strings to build extension of function by epsilon.

    # Build the extension of f by epsilon.
    
    # Add the (key,values) of f_dict
    builder_ext_0=[(k, f_dict[k]) for k in f_dict]

    builder_ext_p2=[(add_n_p_1_to_string(x,n),f_dict[x]+0) for x in list(power_set_str)]

    builder_ext_np1=[(add_n_p_1_to_string("",n),int(0)),]

    
    #Compile together.
    extension_f_dict={**dict(builder_ext_0),**dict(builder_ext_p2),**dict(builder_ext_np1)}
    
    return extension_f_dict

The following function applies get_extension_of_f() and empty_case_get_extension_of_f() to $\mathbb{E}(g)$ for each $g \in \tilde{P}_{n-1}$ and returns $\tilde{P}_{n}$.

In [16]:
def Build_Pn_1(list_df_Pn,power_set_str,n):
    """
    Objective:
    Input:
    Returns:
    """
    # Returns:packets of extensions for each f_dict from Pn, can check those with max size of closed sets (later)
    
    # global power_set_str
    
    P_n1=[]
    for f_dict in list_df_Pn:

        # Store primitive closed sets 

        PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
        dict_PC=dict(PC)
        set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]

        #Get all closed sets for extensions for f_dict
        Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
        Eg.append("empty")

        #Build extensions for f_dict
        extension_of_f_dict=[]
        for term in Eg:
            if term =="empty":
                extension_of_f_dict.append(empty_case_get_extension_of_f(f_dict,term,power_set_str,n))
            else:
                extension_of_f_dict.append(get_extension_of_f(f_dict,term,power_set_str,n))

        #We record packets of extensions where we take +1
        P_n1.append(extension_of_f_dict)
    
    return P_n1

The next function puts the previous information into a dataframe.

In [17]:
def extend_sets_to_Pnp1(P_n1):
    """
    Objective:
    Input:
    Returns:
    """
    #Return the data frame of Pn+1
    
    final_P_n1=[]
    for ls in P_n1:
        final_P_n1=final_P_n1+ls # Adding all the extension packets together.
    df=pd.DataFrame(final_P_n1)
    # df = df.reindex(columns=power_set_str_np1)
    
    return df

Finally get_Pnp1() compiles this information together and allows one to obtain $\tilde{P}_{n-1}$ from $\tilde{P}_{n}$ in a single function.

In [18]:
# This function returns Pn+1
def get_Pnp1(Pn,n):
    """
    Objective:
    Input:
    Returns:
    """

    #inputs
    # Pn and n.

    #New data created
    np1=n+1
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)

    #Load Pn as a list of dictionaries.
    list_df_Pn=Pn.to_dict(orient='records')

    #Create Pn+1 and hold as dataframe.
    P_n1=Build_Pn_1(list_df_Pn,power_set_str,n)
    Pnp1=extend_sets_to_Pnp1(P_n1)

    Pnp1 = Pnp1.reindex(columns=power_set_str_np1)

    inspect_Pn1(Pnp1)

    return Pnp1

### Analysis of $\tilde{P}_{n}$<a name="s114"></a>

The following function returns $|\tilde{P}_n|$ and determines whether $\tilde{P}_n$ is a set.

In [19]:
def inspect_Pn1(df):
    """
    Objective:
    Input:
    Returns:
    """
    print("Number of functions:",len(df.index))
    # df=df.unique()

    duplicate_rows = df[df.duplicated()]
    print("Are there duplicates?",duplicate_rows.shape, "if (0,-) then no common rows")

    # df.head()
    return

The following functions allow one to check every $f \in \tilde{P}_{n}$ are mildly super additive.

In [20]:
def is_valid_function(f,power_set_str):
    """
    Objective: Check if a given function satisfies the properties of a MSA condition
    
    Input:
    f (dict): A dictionary representing the function, where keys are sets in power_set_str and values are integers.
    power_set_str (list of str): A list of string representations of sets.
    
    Returns:
    bool: True if the function satisfies the MSA properties, False otherwise.
    """
    # define the condition to check for each function
    Pn=[string_to_set(x) for x in power_set_str] #convert to sets
    
    # f(T) must equal 0 for elements T in Pn with size 1

    for T in Pn:
        if any(f[set_to_string(T)] != 0 for T in Pn if len(T) == 1):
            return False
    # for any I, J in Pn such that I and J have no common elements, 1 > f(I U J) - f(I) - f(J) or 0 = f(I U J) - f(I) - f(J)
    for I in Pn:
        for J in Pn:
            if len(I.intersection(J)) == 0 and f[set_to_string(I.union(J))] - f[set_to_string(I)] - f[set_to_string(J)] < 0 or f[set_to_string(I.union(J))] - f[set_to_string(I)] - f[set_to_string(J)] > 1:
            # for any I, J in Pn such that I and J have no common elements, 0 <= f(I U J) - f(I) - f(J) <= 1
                return False
    return True

def check_functs_MSA(df,power_set_str):
    """
    Objective: Check a DataFrame of functions for MSA compliance.
    
    Input:
    df (pd.DataFrame): A DataFrame where each row represents a function as a dictionary.
    power_set_str (list of str): A list of string representations of sets.
    
    Returns:
    pd.DataFrame: A DataFrame containing the functions that do not satisfy MSA properties.
    """

    num_rows=df.shape[0]

    bad_f_in_df=[] #indices of functions fail to be MSA in df
    for i in range(0,num_rows):
        f=df.loc[i].to_dict()
        if  is_valid_function(f,power_set_str)==False:
            bad_f_in_df.append(f)
    
    bad_df=pd.DataFrame(bad_f_in_df)
    
    return bad_df #indices of functions that are not MSA

## Cases <a name="s12"></a>

We now determine $\tilde{P}_{n}$ by $\tilde{P}_{n-1}$ for $n \le 5$ recursively using the get_Pnp1() described in Section "Functions",

### $\tilde{P}_{3}$ by $\tilde{P}_{2}$<a name="s121"></a>

It is clear that $\tilde{P}_{2}$ consists of exactly two functions. 

In [21]:
#Easy to know $P_2$
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])

n=2
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P3=get_Pnp1(P2,2)
print(P3)

Number of functions: 10
Are there duplicates? (0, 7) if (0,-) then no common rows
   1  2  3  12  13  23  123
0  0  0  0   0   0   1    1
1  0  0  0   0   1   1    1
2  0  0  0   0   1   0    1
3  0  0  0   0   0   0    1
4  0  0  0   0   0   0    0
5  0  0  0   1   1   1    2
6  0  0  0   1   1   0    1
7  0  0  0   1   1   1    1
8  0  0  0   1   0   1    1
9  0  0  0   1   0   0    1


### $\tilde{P}_{4}$ by $\tilde{P}_{3}$<a name="s122"></a>

In [133]:
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P4=get_Pnp1(P3,3)

Number of functions: 154
Are there duplicates? (0, 15) if (0,-) then no common rows


All the functions in P4 are MSA? Yes

In [30]:
power4=get_powerset_str(4) #powerset on 4 elements
check_functs_MSA(P4,power4).shape
#Yes (0, 0)

(0, 0)

### $\tilde{P}_{5}$ by $\tilde{P}_{4}$<a name="s123"></a>

In [31]:
n=4
power_set_str,power_set_str_np1=get_n_np1_powersets(n) #gives powerset on 4 and 5 elements respectively
P5=get_Pnp1(P4,4)

Number of functions: 10334
Are there duplicates? (0, 31) if (0,-) then no common rows


We check if all functions are MSA: Yes they are

In [32]:
# power5=get_powerset_str(5) #powerset on 4 elements
# check_functs_MSA(P5,power5).shape
# #Yes (0, 0)

(0, 0)

# Decomposition of $|\tilde{P}_n|$ for $n \le 5$  <a name="s2"></a>

As stated in the thesis we now give the sequences $e_{n}$ and $d_{n}$ and state $p_{n+1}=e_n \cdot d_n$. To do so for each  $g \in \tilde{P}_{n-1}$ we record $|\mathbb{E}(g)|$ when we construct $\tilde{P}_{n}$ from $\tilde{P}_{n-1}$.

## Functions <a name="s21"></a>

The following function gets $\mathbb{E}(g)$ similar to get_primitive_for_single_g(df,numb).

In [22]:
def get_Eg_for_single_g(df,numb,power_set_str):

    #Obj: puts Eg into correct format to check if topolgoy
    #Inputs: df=Pn, numb used to get a specific function from df
    
    # Pick function in Pn
    g=df.iloc[numb].to_dict()

    #get primitive closed sets
    PC=[(x,get_barj({x},g)) for x in power_set_str]
    dict_PC=dict(PC)
    set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]

    #Get all closed sets for extensions for g
    Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
    Eg.append(frozenset()) #rembmer the empty set
    return Eg

The following function is similar to get_Eg_for_single_g but instead of specifying a function of Pn with Pn:=df and numb to select position, we take function as a dataframe.

In [23]:
def get_Eg_for_single_g_dfisfunct(funct,power_set_str):
    # print(power_set_str)
    #Obj: puts Eg into correct format to check if topolgoy
    #Inputs: df=Pn, numb used to get a specific function from df
    
    # Pick function in Pn
    g=funct
    
    # print(power_set_str)

    #get primitive closed sets
    PC=[(x,get_barj({x},g)) for x in power_set_str]
    dict_PC=dict(PC)
    set_PC=[sorted_frozenset(get_barj({x},g)) for x in power_set_str]
    # print("len set_PC",len(set_PC))
    
    #Get all closed sets for extensions for g
    Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
    Eg.append(frozenset()) #rembmer the empty set
    return Eg

The following function allows us to record the size of $\mathbb{E}(g)$ for each $f \in \tilde{P}_{n-1}$ by adding the key:value pair of the form 'Eg_Size':$|\mathbb{E}(g)|$ to each $f=g+\epsilon_{I} \in \tilde{P}_{n}$. 

In [24]:
def eg_column_Build_Pn_1(list_df_Pn,power_set_str,n):
    
    """
    Obj:Use Build_Pn_1 and modify change to include Eg size column in functions
    
    Return:
    P_n1: is a list of dataframes considing with f in P_{n+1} and 
    we record the size of E(g) of the g in P_{n-1} which extends to this f
    """
    
    # Returns:packets of extensions for each f_dict from Pn, can check those with max size of closed sets (later)
    
    P_n1=[]
    for index,f_dict in enumerate(list_df_Pn):

        # Store primitive closed sets 
        PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
        dict_PC=dict(PC)
        set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]

        #Get all closed sets for extensions for f_dict
        Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
        Eg.append("empty")
        
        Eg_col_kvpair={"Eg_Size":len(Eg)}

        #Build extensions for f_dict
        extension_of_f_dict=[]
        for term in Eg:
            if term =="empty":                    
                funct=empty_case_get_extension_of_f(f_dict,term,power_set_str,n) #dictionary
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)
            else:
                funct=get_extension_of_f(f_dict,term,power_set_str,n)
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)

        #We record packets of extensions where we take +1
        P_n1.append(extension_of_f_dict)
    
    return P_n1

The following function take the previous list of dataframes for $\tilde{P}_{n}$ with the column 'Eg_Size':$|\mathbb{E}(g)|$.

In [25]:
def get_Eg_column_Pn(df):
    df=df.copy()
    
    #Get Eg for the row in df.

    # Create a new column called "new_column"
    #--------
    power_set_str_n=get_powerset_str(n+1)
    # print("Power",power_set_str_n)
    # print("Current Pn-1 \n",df)

    # Insert data into the new column
    for i, row in df.iterrows():   
        funct=dict(row)
        funct.pop('Eg_Size', None)
        # print(funct)
        Eg=get_Eg_for_single_g_dfisfunct(funct,power_set_str_n)
        # print(Eg)
        df.at[i, 'Eg_Size'] = int(len(Eg))
        # print(len(Eg))
    return df

We repeat the above process but for the dataframe consisting of a subset of $\tilde{P}_{n-1}$. 

In [26]:
def get_Pnp1_for_single_funct(Pn,n):
    
    #inputs
    # Pn:dataframe and n.
    
    """
    Returns a dataframe of extensions in Pn+1 for dataframe of functions in Pn.
    """
    """
    #Example
    n=3
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)
    # P3

    funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
    Pn=pd.DataFrame(funct_list)

    #The following should be functions in P4 that are extendions of functions in funct_list
    f_P3=get_Pnp1_for_single_funct(Pn,n)
    print(f_P3)
    """

    #New data created
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)

    #Load Pn as a list of dictionaries.
    list_df_Pn=Pn.to_dict(orient='records')

    #Create Pn+1 and hold as dataframe.
    P_n1=eg_column_Build_Pn_1(list_df_Pn,power_set_str,n)
    Pnp1=extend_sets_to_Pnp1(P_n1) #<---- ISSUE
    
    # power_set_str_np1
    
    columns=list(power_set_str_np1)+["Eg_Size"]
    Pnp1 = Pnp1.reindex(columns=columns)


    inspect_Pn1(Pnp1)

    return Pnp1

We give an application of the function get_Pnp1_for_single_funct() in the following example. Consider a function in $\tilde{P}_{3}$.

In [159]:
#Get P3
# n=3
# power_set_str,power_set_str_np1=get_n_np1_powersets(n)

P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])
n=2
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P3=get_Pnp1(P2,2)
print(P3)

Number of functions: 10
Are there duplicates? (0, 7) if (0,-) then no common rows
   1  2  3  12  13  23  123
0  0  0  0   0   1   0    1
1  0  0  0   0   1   1    1
2  0  0  0   0   0   0    1
3  0  0  0   0   0   1    1
4  0  0  0   0   0   0    0
5  0  0  0   1   1   0    1
6  0  0  0   1   1   1    2
7  0  0  0   1   0   1    1
8  0  0  0   1   1   1    1
9  0  0  0   1   0   0    1


In [27]:
# You can take subsets of rows as a database, then extend. Better than below
funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
Pn=pd.DataFrame(funct_list)
print("The functions we are extending")
print(Pn)

The functions we are extending
   1  2  3  12  23  13  123
0  0  0  0   0   0   0    0


We see that there are $19=|\mathbb{E}(g)|$ extensions of this function in $\tilde{P}_{4}$. Do not conflate Eg_size in the dataframe below with how we $e_n$ and $d_n$.

In [28]:
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
#The following should be functions in P4 that are extendions of functions in funct_list
f_P3=get_Pnp1_for_single_funct(Pn,n)
print(f_P3)

Number of functions: 19
Are there duplicates? (0, 16) if (0,-) then no common rows
    1  2  3  4  12  13  14  23  24  34  123  124  134  234  1234  Eg_Size
0   0  0  0  0   0   0   0   0   0   0    0    1    1    0     1       19
1   0  0  0  0   0   0   0   0   1   0    0    1    0    1     1       19
2   0  0  0  0   0   0   1   0   1   1    0    1    1    1     1       19
3   0  0  0  0   0   0   0   0   0   0    0    1    0    0     1       19
4   0  0  0  0   0   0   1   0   1   0    0    1    1    1     1       19
5   0  0  0  0   0   0   0   0   0   0    0    0    1    0     1       19
6   0  0  0  0   0   0   0   0   0   1    0    1    1    1     1       19
7   0  0  0  0   0   0   0   0   1   1    0    1    1    1     1       19
8   0  0  0  0   0   0   0   0   0   0    0    0    0    1     1       19
9   0  0  0  0   0   0   0   0   0   0    0    0    1    1     1       19
10  0  0  0  0   0   0   0   0   0   0    0    1    0    1     1       19
11  0  0  0  0   0   0   0   

## Cases <a name="s22"></a>

Recall the following.


Let $(p_{n})_{n \in \mathbb{N}}$ denote the sequence of terms $p_{n}:=|\tilde{P}_n|$. Let $\boldsymbol{e}_{n}:=(e_{n,i})$ denote the increasing sequence of distinct terms $|\mathbb{E}(g)|$ for $g \in \tilde{P}_{n}$, and $\boldsymbol{d}_{n}:=(d_{n,i})$ denote the sequence where $d_{n,i}$ denotes the number of $g \in \tilde{P}_{n}$ which give the value $e_{n,i}$.

  $$p_{n+1} =\boldsymbol{e}_n \cdot \boldsymbol{d}_{n}=\sum e_{n,i}
 \times d_{n,i}$$

### Decomposition of $|\tilde{P}_3|$<a name="s221"></a>

We consider the extension of $\tilde{P}_{2}$ to $\tilde{P}_{3}$ and record the size of each $|\mathbb{E}(g)|$ for $ \in \tilde{P}_{2}$. We then obtain the sequences $e_{2}$ and $d_{2}$ (possiblly not ordered by increasing size), and state

$$p_3=e_2 \cdot d_2$$

First get $\tilde{P}_2$.

In [163]:
n=1
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])

In [164]:
power_set_str=get_powerset_str(2)

In [165]:
P2_eg=get_Eg_column_Pn(P2)
P2_eg=P2_eg.sort_values('Eg_Size') 

df=P2_eg
print(f"e_n : d_n")
print(df['Eg_Size'].value_counts()) #count number of 13s and 19s

e_n : d_n
5.0    2
Name: Eg_Size, dtype: int64


In [166]:
print(P2_eg)

   1  2  12  Eg_Size
0  0  0   0      5.0
1  0  0   1      5.0


The decomposition of $p_3=5\cdot1 +5 \cdot 1$.

### Decomposition of $|\tilde{P}_4|$<a name="s222"></a>

We consider the extension of $\tilde{P}_{3}$ to $\tilde{P}_{4}$ and record the size of each $|\mathbb{E}(g)|$ for $ \in \tilde{P}_{3}$. We then obtain the sequences $e_{3}$ and $d_{3}$  and state

$$p_4=e_3 \cdot d_3.$$

First get $\tilde{P}_3$.

In [167]:
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])
n=2
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P3=get_Pnp1(P2,2)
print(P3)

Number of functions: 10
Are there duplicates? (0, 7) if (0,-) then no common rows
   1  2  3  12  13  23  123
0  0  0  0   0   1   0    1
1  0  0  0   0   1   1    1
2  0  0  0   0   0   0    1
3  0  0  0   0   0   1    1
4  0  0  0   0   0   0    0
5  0  0  0   1   1   0    1
6  0  0  0   1   1   1    2
7  0  0  0   1   0   1    1
8  0  0  0   1   1   1    1
9  0  0  0   1   0   0    1


In [152]:
power_set_str=get_powerset_str(3)

P3_eg=get_Eg_column_Pn(P3)
P3_eg=P3_eg.sort_values('Eg_Size') # get those 13,then 19

df=P3_eg
print(f"e_n : d_n")
print(df['Eg_Size'].value_counts()) #count number of 13s and 19s

e_n : d_n
13.0    6
19.0    4
Name: Eg_Size, dtype: int64


In particular $p_4 = 13 \cdot 6 + 19 \cdot 4$.

For example we can extract exactly all those function which have $|\mathbb{E}(g)|=13$ 

In [153]:
y=P3_eg[P3_eg.Eg_Size == 13]
print(y)
print(y.shape)

   1  2  3  12  13  23  123  Eg_Size
0  0  0  0   0   1   0    1     13.0
1  0  0  0   0   1   1    1     13.0
3  0  0  0   0   0   1    1     13.0
5  0  0  0   1   1   0    1     13.0
7  0  0  0   1   0   1    1     13.0
9  0  0  0   1   0   0    1     13.0
(6, 8)


#### Remark

Similar to section "4) Example $\mathbb{E}(g)$ for $g \in  \tilde{P}_{3}$" we can obtain $E(g)$ for a fixed function.

In [88]:
#Get Eg for the function in P3
#3  0  0  0   0   0   0    1      19
numb =3
df=P3
g=df.iloc[numb].to_dict()
print(g,"\n")

EEg_example=get_Eg_for_single_g(df,numb,power_set_str)
EEg_example

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 1, '123': 1} 



[frozenset({'1', '12', '123', '13', '2', '23', '3'}),
 frozenset({'13'}),
 frozenset({'12', '123', '13', '2', '3'}),
 frozenset({'13', '3'}),
 frozenset({'12', '2'}),
 frozenset({'12', '123', '13', '2', '23', '3'}),
 frozenset({'12'}),
 frozenset({'12', '13'}),
 frozenset({'12', '13', '3'}),
 frozenset({'1', '12', '123', '13', '2', '3'}),
 frozenset({'12', '13', '2'}),
 frozenset({'12', '13', '2', '3'}),
 frozenset()]

### Decomposition of $|\tilde{P}_5|$<a name="s223"></a>

We consider the extension of $\tilde{P}_{4}$ to $\tilde{P}_{5}$ and record the size of each $|\mathbb{E}(g)|$ for $ \in \tilde{P}_{4}$. We then obtain the sequences $e_{4}$ and $d_{4}$ (possiblly not ordered by increasing size), and state

$$p_5=e_4 \cdot d_4$$

First get $\tilde{P}_4$.

In [91]:
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P4=get_Pnp1(P3,3)

Number of functions: 154
Are there duplicates? (0, 15) if (0,-) then no common rows


In [92]:
power_set_str=get_powerset_str(4)

P4_eg=get_Eg_column_Pn(P4)
P4_eg=P4_eg.sort_values('Eg_Size') # get those 13,then 19


df=P4_eg
print(f"e_n : d_n")
print(df['Eg_Size'].value_counts()) #count number of 13s and 19s

e_n : d_n
42.0     24
45.0     24
47.0     24
82.0     16
46.0     12
54.0     12
111.0    12
99.0     10
69.0      8
133.0     8
167.0     4
Name: Eg_Size, dtype: int64


$p_5 = 42 · 24 + 45 · 24 + 46 · 12 + 47 · 24 + 54 · 12 + 69 · 8 + 82 · 16 + 99 · 10 + 111 · 12 + 133 · 8 + 167 · 4$

# Which $g \in \tilde{P}_{n}$ attain max and min $|\mathbb{E}(g)|$<a name="s3"></a>

Let $(p_{n})_{n \in \mathbb{N}}$ denote the sequence of terms $p_{n}:=|\tilde{P}_n|$, and let $\boldsymbol{e}_{n}:=(e_{n,i})$ denote the increasing sequence of distinct terms $|\mathbb{E}(g)|$ for $g \in \tilde{P}_{n}$.

Recall Section $5.2$ of the thesis. In particular we have the sequences $\mathcal{U}=(U_n)$ and $\mathcal{L}=(L_n)$ defined by the maximal and minimal term of $e_n$.

In this section we ask which $g \in \tilde{P}_{n}$ attain $U_n$ and $L_n$ (there is more than one in each case). In particular we would like to know those examples which have a particularly nice form.

## $U_3$ and $z \in \tilde{P_n}$<a name="s31"></a>

The all zero function $z\in \tilde{P_n}$ we conjecture always attains $U_n=|\mathbb{E}(z)|$.

Consider {'1': 0,'2': 0,"12":0} in $\tilde{P}_2$, this obtaiin $U_2$.

In [37]:
n=1
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])

#one function selected
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0}])

power_set_str=get_powerset_str(2)
P2_eg=get_Eg_column_Pn(P2)
P2_eg=P2_eg.sort_values('Eg_Size') 

df=P2_eg
print(f"e_n : d_n")
print(df['Eg_Size'].value_counts()) #count number of 13s and 19s

e_n : d_n
5.0    1
Name: Eg_Size, dtype: int64


In [38]:
print(P2_eg)

   1  2  12  Eg_Size
0  0  0   0      5.0


The all zero function $z$ is a good candidate as it also has maximal $|\mathbb{E}(z)|$ in $P_{5}$ (determined separately)

In [ ]:
power_set_str=get_powerset_str(5)
#Find index of zero function
# Take a g in P5 and calculate Eg
# P5.iloc[1].to_dict()
all_zero_funct={'1': 0,'2': 0,'3': 0,'4': 0,'5': 0,'12': 0,'13': 0,'14': 0,'15': 0,'23': 0,'24': 0,'25': 0,'34': 0,'35': 0,'45': 0,'123': 0,'124': 0,'125': 0,'134': 0,'135': 0,'145': 0,'234': 0,'235': 0,'245': 0,'345': 0,'1234': 0,'1235': 0,'1245': 0,'1345': 0,'2345': 0,'12345': 0}
df=P5
my_dict=all_zero_funct
index = df.loc[df.eq(my_dict).all(1)].index[0]
print(index)#5166 #works
# df.loc[5166]#
# Get Eg for this. #Should be maximum
# Eg_zeroP5=get_Eg_for_single_g(df,index,power_set_str)
# print(len(Eg_zeroP5)) #7580

Consider $z \in \tilde{P}_n$, what other functions also give $U_n$? 

Simplist case to consider is $\tilde{P}_3$. We determine $f \in \tilde{P}_3$ and determine $\mathbb{E}(f)$ for each, then extract those $f$ such that we have
$|\mathbb{E}(f)|=U_3$.

In [60]:
#get P3
P2=pd.DataFrame([{'1': 0, '2': 0, '12': 0},{'1': 0, '2': 0, '12': 1}])
n=2
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
P3=get_Pnp1(P2,2)
print(P3)

Number of functions: 10
Are there duplicates? (0, 7) if (0,-) then no common rows
   1  2  3  12  13  23  123
0  0  0  0   0   0   1    1
1  0  0  0   0   1   1    1
2  0  0  0   0   1   0    1
3  0  0  0   0   0   0    1
4  0  0  0   0   0   0    0
5  0  0  0   1   1   1    2
6  0  0  0   1   1   0    1
7  0  0  0   1   1   1    1
8  0  0  0   1   0   1    1
9  0  0  0   1   0   0    1


In [61]:
power_set_str=get_powerset_str(3)

P3_eg=get_Eg_column_Pn(P3)
P3_eg=P3_eg.sort_values('Eg_Size') # get those 13,then 19

df=P3_eg
print(f"e_n : d_n")
print(df['Eg_Size'].value_counts()) #count number of 13s and 19s

e_n : d_n
13.0    6
19.0    4
Name: Eg_Size, dtype: int64


There are $4$ which give $U_3$.

In [115]:
y=P3_eg[P3_eg.Eg_Size == 19]
print(y)
print(y.shape)
#Get Eg for the function in P3
#3  0  0  0   0   0   0    1      19

   1  2  3  12  13  23  123  Eg_Size
3  0  0  0   0   0   0    1     19.0
4  0  0  0   0   0   0    0     19.0
5  0  0  0   1   1   1    2     19.0
7  0  0  0   1   1   1    1     19.0
(4, 8)


Remove "Eg_Size" column and turn to a list of dictionaries.

In [116]:
y.drop('Eg_Size', axis=1, inplace=True)
y

C:\Users\RhysL\AppData\Local\Temp\ipykernel_12024\1174555318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.drop('Eg_Size', axis=1, inplace=True)


,1,2,3,12,13,23,123
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0
5,0,0,0,1,1,1,2
7,0,0,0,1,1,1,1


In [123]:
df=y
list_of_dicts = df.to_dict(orient='records')
list_of_dicts

[{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 1},
 {'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 0},
 {'1': 0, '2': 0, '3': 0, '12': 1, '13': 1, '23': 1, '123': 2},
 {'1': 0, '2': 0, '3': 0, '12': 1, '13': 1, '23': 1, '123': 1}]

We now determine $\mathbb{E}(g)$ for each of the above functions. We then ask is $\mathbb{E}(g)$ are equal. Recall get_Eg_for_single_g_dfisfunct(). 


As the order of the output of get_Eg_for_single_g_dfisfunct() can vary we sort everything into a standard format

In particular we pick the $f \in \tilde{P}_{3}$  which obtain $|\mathbb{E}(f)|=19=U_3$ and $\mathbb{E}(f)$.

In [125]:
# Eg_cases=[]
sorted_Eg_cases = []

for g in list_of_dicts:
    print(g)
    #[{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 1},
    EEg_example=get_Eg_for_single_g_dfisfunct(g,power_set_str)
 # [frozenset({'12'}), frozenset({'2', '23', '13', '12', '1'}), frozenset({'2', '23', '3', '13', '12', '1', '123'}), 
 #  frozenset({'2', '23', '3', '13', '12', '1'}), frozenset({'23'}), frozenset({'13', '12', '2', '23'}),
 #  frozenset({'1', '13', '12'}), frozenset({'13', '12'}), frozenset({'12', '23'}), frozenset({'2', '23', '3', '13', '12'}), 
 #  frozenset({'13', '23'}), frozenset({'13', '12', '23', '3'}), frozenset({'23', '3', '13', '12', '1'}), frozenset({'1', '13', '12', '23'}), 
 #  frozenset({'13', '12', '23'}), frozenset({'13', '23', '3'}), frozenset({'13'}), frozenset({'12', '2', '23'}), frozenset()] 
    
    e=EEg_example
    sorted_e=[] #elements are
    # [['12'], ['1', '12', '13', '2', '23'], ['1', '12', '123', '13', '2', '23', '3'], ['1', '12', '13', '2', '23', '3'],
    #  ['23'], ['12', '13', '2', '23'], ['1', '12', '13'], ['12', '13'], ['12', '23'], ['12', '13', '2', '23', '3'], ['13', '23'],
    #  ['12', '13', '23', '3'], ['1', '12', '13', '23', '3'], 
    #  ['1', '12', '13', '23'], ['12', '13', '23'], ['13', '23', '3'], ['13'], ['12', '2', '23'], []]
    for fs in e:
        sorted_fs=sorted(list(fs))
        sorted_e.append(sorted_fs)
        
    sorted_Eg_cases.append(sorted_e)
    print(sorted_e,"\n")

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 1}
[['12'], ['1', '12', '13', '2', '23'], ['1', '12', '123', '13', '2', '23', '3'], ['1', '12', '13', '2', '23', '3'], ['23'], ['12', '13', '2', '23'], ['1', '12', '13'], ['12', '13'], ['12', '23'], ['12', '13', '2', '23', '3'], ['13', '23'], ['12', '13', '23', '3'], ['1', '12', '13', '23', '3'], ['1', '12', '13', '23'], ['12', '13', '23'], ['13', '23', '3'], ['13'], ['12', '2', '23'], []] 

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 0}
[['12', '123', '13'], ['12', '123', '2', '23'], ['1', '12', '123', '13', '2', '23', '3'], ['12', '123'], ['1', '12', '123', '13', '2', '23'], ['123', '13'], ['12', '123', '13', '23', '3'], ['12', '123', '13', '2', '23', '3'], ['123', '23'], ['123', '13', '23'], ['12', '123', '23'], ['12', '123', '13', '23'], ['1', '12', '123', '13', '23'], ['123', '13', '23', '3'], ['12', '123', '13', '2', '23'], ['1', '12', '123', '13', '23', '3'], ['1', '12', '123', '13'], ['123'], []] 

{'1': 

We see that not all $\mathbb{E}(g)$ are equal. Let $f \in \tilde{P}_n$ do we always have a surjection  $\mathbb{E}(z) \rightarrow \mathbb{E}(f)$ (no otherwise these would be equal).

What is $\mathbb{E}(z)$. 

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 0}
[['12', '123', '13'], ['12', '123', '2', '23'], ['1', '12', '123', '13', '2', '23', '3'], ['12', '123'], ['1', '12', '123', '13', '2', '23'], ['123', '13'], ['12', '123', '13', '23', '3'], ['12', '123', '13', '2', '23', '3'], ['123', '23'], ['123', '13', '23'], ['12', '123', '23'], ['12', '123', '13', '23'], ['1', '12', '123', '13', '23'], ['123', '13', '23', '3'], ['12', '123', '13', '2', '23'], ['1', '12', '123', '13', '23', '3'], ['1', '12', '123', '13'], ['123'], []] 

## $L_3$ <a name="s32"></a>

Lets pick the $f \in \tilde{P}_{3}$  which obtain $|\mathbb{E}(f)|=13=L_3$. In this section we obtain to $f$ and the $\mathbb{E}(f)$.

In [126]:
y=P3_eg[P3_eg.Eg_Size == 13]
print(y)
print(y.shape)
#Get Eg for the function in P3
#3  0  0  0   0   0   0    1      19

   1  2  3  12  13  23  123  Eg_Size
0  0  0  0   0   0   1    1     13.0
1  0  0  0   0   1   1    1     13.0
2  0  0  0   0   1   0    1     13.0
6  0  0  0   1   1   0    1     13.0
8  0  0  0   1   0   1    1     13.0
9  0  0  0   1   0   0    1     13.0
(6, 8)


In [127]:
y.drop('Eg_Size', axis=1, inplace=True)
# y
df=y
list_of_dicts = df.to_dict(orient='records')
list_of_dicts

C:\Users\RhysL\AppData\Local\Temp\ipykernel_12024\1624874608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.drop('Eg_Size', axis=1, inplace=True)


[{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 1, '123': 1},
 {'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
 {'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '123': 1},
 {'1': 0, '2': 0, '3': 0, '12': 1, '13': 1, '23': 0, '123': 1},
 {'1': 0, '2': 0, '3': 0, '12': 1, '13': 0, '23': 1, '123': 1},
 {'1': 0, '2': 0, '3': 0, '12': 1, '13': 0, '23': 0, '123': 1}]

In [128]:
# Eg_cases=[]
sorted_Eg_cases = []

for g in list_of_dicts:
    print(g)
# {'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 1, '123': 1}
    EEg_example=get_Eg_for_single_g_dfisfunct(g,power_set_str)

    
    e=EEg_example
    sorted_e=[] #elements are
  # [['12'], ['12', '123', '13', '2', '3'], ['12', '13', '2', '3'], ['13', '3'], ['12', '13', '3'], ['1', '12', '123', '13', '2', '23', '3'], ['12', '13', '2'], ['12', '2'], ['12', '123', '13', '2', '23', '3'], ['12', '13'], ['1', '12', '123', '13', '2', '3'], ['13'], []] 
    for fs in e:
        sorted_fs=sorted(list(fs))
        sorted_e.append(sorted_fs)
        
    sorted_Eg_cases.append(sorted_e)
    print(sorted_e,"\n")

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 1, '123': 1}
[['12'], ['12', '123', '13', '2', '3'], ['12', '13', '2', '3'], ['13', '3'], ['12', '13', '3'], ['1', '12', '123', '13', '2', '23', '3'], ['12', '13', '2'], ['12', '2'], ['12', '123', '13', '2', '23', '3'], ['12', '13'], ['1', '12', '123', '13', '2', '3'], ['13'], []] 

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1}
[['12', '3'], ['12'], ['1', '12', '123', '2', '3'], ['12', '123', '2', '23', '3'], ['1', '12', '123', '13', '2', '23', '3'], ['1', '12', '123', '2', '23', '3'], ['12', '123', '3'], ['12', '123', '2', '3'], ['1', '12', '123', '13', '2', '3'], ['3'], ['1', '12', '123', '13', '3'], ['1', '12', '123', '3'], []] 

{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '123': 1}
[['12', '23', '3'], ['23', '3'], ['12'], ['1', '12', '123', '13', '2', '23', '3'], ['23'], ['1', '12', '23', '3'], ['1', '12', '123', '23', '3'], ['1', '12', '123', '2', '23', '3'], ['1', '12'], ['1', '12', '123', '13', '23', '3'], 

We conjecture that:
    
    {'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1}
[['12', '3'], ['12'], ['1', '12', '123', '2', '3'], ['12', '123', '2', '23', '3'], ['1', '12', '123', '13', '2', '23', '3'], ['1', '12', '123', '2', '23', '3'], ['12', '123', '3'], ['12', '123', '2', '3'], ['1', '12', '123', '13', '2', '3'], ['3'], ['1', '12', '123', '13', '3'], ['1', '12', '123', '3'], []] 

Generalises to a function which always gives $L_{n}$.

# $\tilde{P}_6$ and decomposition of $|\tilde{P}_6|$ (Debt)<a name="s4"></a>

The following code is an application of previous cases. I leave this section as debt. I f you wish to understand the details email me and i will fill organise it for you then

As P6=get_Pnp1(P5,5) takes too long we break the construction into parts. During this process we record the number of extension each function in P5 has, this will be used to give the sum

$$|P_6|=\sum_{g \in P_5} |\rho^{-1}(g)|$$

## Functions <a name="s21"></a>

In [25]:
def get_Eg_column_Pn(df):
    df=df.copy()

    # Create a new column called "new_column"
    df['Eg_Size'] = None

    # Insert data into the new column
    for i, row in df.iterrows():    
        Eg=get_Eg_for_single_g(df,i,power_set_str)
        df.at[i, 'Eg_Size'] = int(len(Eg))

    return df

In [25]:
def eg_column_Build_Pn_1(list_df_Pn,power_set_str,n):
    
    """
    Obj:Use Build_Pn_1 and modify change to include Eg size column in functions
    """
    
    # Returns:packets of extensions for each f_dict from Pn, can check those with max size of closed sets (later)
    
    P_n1=[]
    for index,f_dict in enumerate(list_df_Pn):

        # Store primitive closed sets 
        PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
        dict_PC=dict(PC)
        set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]

        #Get all closed sets for extensions for f_dict
        Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
        Eg.append("empty")
        
        Eg_col_kvpair={"Eg_Size":len(Eg)}

        #Build extensions for f_dict
        extension_of_f_dict=[]
        for term in Eg:
            if term =="empty":                    
                funct=empty_case_get_extension_of_f(f_dict,term,power_set_str,n) #dictionary
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)
            else:
                funct=get_extension_of_f(f_dict,term,power_set_str,n)
                funct.update(Eg_col_kvpair)
                extension_of_f_dict.append(funct)

        #We record packets of extensions where we take +1
        P_n1.append(extension_of_f_dict)
    
    return P_n1

In [137]:
def get_Pnp1_for_single_funct(Pn,n):
    
    #inputs
    # Pn:dataframe and n.
    
    """
    Returns a dataframe of extensions in Pn+1 for dataframe of functions in Pn.
    """
    """
    #Example
    n=3
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)
    # P3

    funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
    Pn=pd.DataFrame(funct_list)

    #The following should be functions in P4 that are extendions of functions in funct_list
    f_P3=get_Pnp1_for_single_funct(Pn,n)
    print(f_P3)
    """

    #New data created
    power_set_str,power_set_str_np1=get_n_np1_powersets(n)

    #Load Pn as a list of dictionaries.
    list_df_Pn=Pn.to_dict(orient='records')

    #Create Pn+1 and hold as dataframe.
    P_n1=eg_column_Build_Pn_1(list_df_Pn,power_set_str,n)
    Pnp1=extend_sets_to_Pnp1(P_n1) #<---- ISSUE
    
    power_set_str_np1
    
    columns=list(power_set_str_np1)+["Eg_Size"]
    Pnp1 = Pnp1.reindex(columns=columns)


    inspect_Pn1(Pnp1)

    return Pnp1

In [25]:
def Pnp1_from_Pn_part(df,n,calc_start,calc_end):
    # df where we are extending from
    # calc_start=0
    # calc_end=5

    rows=range(calc_start,calc_end) #What functions we are extending
    Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

    # print("The functions we are extending")
    # print(Pn,"\n")

    part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
    # print(part_Pn.shape)
    
    strt=f"P{n+1}_parts\{calc_start}_{calc_end}_Part_fromP{n}.xlsx"
    part_Pn.to_excel(strt) #Stores in excel
    return

## Example of functions used and process for P3 <a name="s22"></a>

### Single function extension example

Here we see get_Pnp1_for_single_funct in action for P3.

In [138]:
#Testing functions example
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
# P3

# You can take subsets of rows as a database, then extend. Better than below
funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
Pn=pd.DataFrame(funct_list)
print("The functions we are extending")
print(Pn)

#The following should be functions in P4 that are extendions of functions in funct_list
f_P3=get_Pnp1_for_single_funct(Pn,n)
print(f_P3)

The functions we are extending
   1  2  3  12  23  13  123
0  0  0  0   0   0   0    0
Number of functions: 19
Are there duplicates? (0, 16) if (0,-) then no common rows
    1  2  3  4  12  13  14  23  24  34  123  124  134  234  1234  Eg_Size
0   0  0  0  0   0   0   1   0   1   1    0    1    1    1     1       19
1   0  0  0  0   0   0   0   0   0   0    0    1    0    1     1       19
2   0  0  0  0   0   0   0   0   0   0    0    0    1    1     1       19
3   0  0  0  0   0   0   0   0   0   0    0    1    1    1     1       19
4   0  0  0  0   0   0   1   0   1   0    0    1    1    1     1       19
5   0  0  0  0   0   0   0   0   1   1    0    1    1    1     1       19
6   0  0  0  0   0   0   1   0   0   0    0    1    1    1     1       19
7   0  0  0  0   0   0   0   0   0   0    0    0    0    1     1       19
8   0  0  0  0   0   0   0   0   0   0    0    1    1    0     1       19
9   0  0  0  0   0   0   0   0   0   0    0    0    0    0     1       19
10  0  0  0  0  

### Building P4 from P3 example

In [ ]:
n=3
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
# P3
df=P3 #Total dataframe

In [ ]:
calc_start=0
calc_end=5

rows=range(calc_start,calc_end) #What functions we are extending
Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

print("The functions we are extending")
print(Pn,"\n")

part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
print(part_Pn)
strt=f"P3_parts_example\{calc_start}_{calc_end}_PartP4.xlsx"
part_Pn.to_excel(strt) #Stores in excel

In [ ]:
calc_start=5
calc_end=10

rows=range(calc_start,calc_end) #What functions we are extending
Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

print("The functions we are extending")
print(Pn,"\n")

part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
print(part_Pn)
strt=f"P3_parts_example\{calc_start}_{calc_end}_PartP4.xlsx"
part_Pn.to_excel(strt) #Stores in excel

 We now merge the parts together.

In [ ]:
P4_part1=pd.read_excel('P3_parts_example/0_5_PartP4.xlsx', index_col=0) 
P4_part2=pd.read_excel('P3_parts_example/5_10_PartP4.xlsx', index_col=0) 

merged_df = pd.concat([P4_part1, P4_part2], ignore_index=True)

merged_df

## Building P6 from P5 <a name="s23"></a>

You can run the following cell to get the data for P6 (take about 4hrs)

In [19]:
# Part builder cell for P5 to P6 #At this rate it will take 1.43hrs<time<6hrs to run this calculation

n=5
power_set_str,power_set_str_np1=get_n_np1_powersets(n)
df=P5 #Total dataframe

### Test case

In [20]:
start_perf,start_process = time.perf_counter(),time.process_time()
Pnp1_from_Pn_part(df,n,0,100)
end_perf,end_process = time.perf_counter(),time.process_time()

print(f"Perf timer {end_perf-start_perf}") #This method returns the time in seconds.
print(f"Process timer {end_process-start_process}") #measures the time the process takes, including time that the process is blocked

Number of functions: 47417
Are there duplicates? (0, 64) if (0,-) then no common rows
Perf timer 107.10247100000004
Process timer 73.859375


### Running to construct P6

Constructs a series of 101 xlxs files to store dataframe of parts of P6

In [21]:
def runner(start,end): 
    start_perf,start_process = time.perf_counter(),time.process_time()
    Pnp1_from_Pn_part(df,n,start,end)
    end_perf,end_process = time.perf_counter(),time.process_time()

    print(f"Perf timer {end_perf-start_perf}") #This method returns the time in seconds.
    print(f"Process timer {end_process-start_process}") #measures the time the process takes, including time that the process is blocked
    return

In [ ]:
for i in range(0, 10300, 100):
    start=i
    end=start+100
    runner(start,end)

In [23]:
calc_start=10300
calc_end=10334

rows=range(calc_start,calc_end) #What functions we are extending
Pn=df.iloc[rows] #takes database of rows 0,1..,calc_up_to

part_Pn=get_Pnp1_for_single_funct(Pn,n) #Extensions of Pn
strt=f"P6_parts\{calc_start}_{calc_end}_Part_fromP5.xlsx"
part_Pn.to_excel(strt) #Stores in excel

Number of functions: 13087
Are there duplicates? (0, 64) if (0,-) then no common rows


## Analyse P6 <a name="s24"></a>

Here we obtain $|\tilde{P}_{6}|$.

To do so we load 

By loading all files in P6_parts and taking the shape.
As part of this calculation we can also decompose this number into the sum,

$$|P_6|=\sum_{g \in P_5} |\rho^{-1}(g)|\cdot 1_{g}$$

The following loads the data for the parts of $P6$ (takes about 1.5hrs)

In [4]:
#Store number of rows for each data packet of P6
P6_size_list=[]
Eg_numb_functs_init_counter=[]

#Main Body of calculations
for i in range(0, 10300, 100):
    #Stores number for this packet in P6_size_list
    start=i
    end=start+100
    P6_part=pd.read_excel(f'P6_parts/{start}_{end}_Part_fromP5.xlsx', index_col=0)
    print(f'P6_parts/{start}_{end}_Part_fromP5.xlsx')
    row_numb=P6_part.shape[0]
    P6_size_list.append(row_numb)

    # In this packet gets Eg_Size and get counter for eg sizes and number of functions in Eg_numb_functs_init_counter
    values = P6_part['Eg_Size'].value_counts(ascending=True).keys().tolist()
    counts = P6_part['Eg_Size'].value_counts(ascending=True).tolist()
    zipped=list(zip(values,counts))
    Eg_numb_functs_init_counter.extend(zipped)
    
# It remains to do 10300-10334
#Stores number for this packet in P6_size_list
start,end=(10300,10334)
P6_part=pd.read_excel(f'P6_parts/{start}_{end}_Part_fromP5.xlsx', index_col=0)
print(f'P6_parts/{start}_{end}_Part_fromP5.xlsx')
row_numb=P6_part.shape[0]
P6_size_list.append(row_numb)

# In this packet gets Eg_Size and get counter for eg sizes and number of functions in Eg_numb_functs_init_counter
values = P6_part['Eg_Size'].value_counts(ascending=True).keys().tolist()
counts = P6_part['Eg_Size'].value_counts(ascending=True).tolist()
zipped=list(zip(values,counts))
Eg_numb_functs_init_counter.extend(zipped)

#output-------------------------------------------------------------------------

#Final answer
P6_size=sum(P6_size_list)
print(f"There are {P6_size} functions in P6 \n")

P6_parts/0_100_Part_fromP5.xlsx
P6_parts/100_200_Part_fromP5.xlsx
P6_parts/200_300_Part_fromP5.xlsx
P6_parts/300_400_Part_fromP5.xlsx
P6_parts/400_500_Part_fromP5.xlsx
P6_parts/500_600_Part_fromP5.xlsx
P6_parts/600_700_Part_fromP5.xlsx
P6_parts/700_800_Part_fromP5.xlsx
P6_parts/800_900_Part_fromP5.xlsx
P6_parts/900_1000_Part_fromP5.xlsx
P6_parts/1000_1100_Part_fromP5.xlsx
P6_parts/1100_1200_Part_fromP5.xlsx
P6_parts/1200_1300_Part_fromP5.xlsx
P6_parts/1300_1400_Part_fromP5.xlsx
P6_parts/1400_1500_Part_fromP5.xlsx
P6_parts/1500_1600_Part_fromP5.xlsx
P6_parts/1600_1700_Part_fromP5.xlsx
P6_parts/1700_1800_Part_fromP5.xlsx
P6_parts/1800_1900_Part_fromP5.xlsx
P6_parts/1900_2000_Part_fromP5.xlsx
P6_parts/2000_2100_Part_fromP5.xlsx
P6_parts/2100_2200_Part_fromP5.xlsx
P6_parts/2200_2300_Part_fromP5.xlsx
P6_parts/2300_2400_Part_fromP5.xlsx
P6_parts/2400_2500_Part_fromP5.xlsx
P6_parts/2500_2600_Part_fromP5.xlsx
P6_parts/2600_2700_Part_fromP5.xlsx
P6_parts/2700_2800_Part_fromP5.xlsx
P6_parts/2800

We get Eg_numb_functs_init_counter as an output also this records the decomposition of the sum $|P6|$. Eg_numb_functs_init_counter consists of (x,y) where x=|Eg| and y= x \cdot the number of times x appeared in that packet. Therefore it is neccsary to sum to get (x,Y) where Y isx \cdot the number of times x appeared in all packets. To get the number of times x appears it is necessary to divide Y by x.

In [38]:
#He now obtain Eg_numb_functs from Eg_numb_functs_init_counter

# This is a List of tuples where have  joined e.g (13, 6),(13, 19) to (13, 25) from Eg_numb_functs_init_counter

from collections import defaultdict

sums = defaultdict(int)

for first, second in Eg_numb_functs_init_counter:
    sums[first] += second

Eg_numb_functs = [(first, second) for first, second in sums.items()]
Eg_numb_functs = sorted(Eg_numb_functs, key=lambda x: x[0])#order so in increasing with respect to eg size

#Build a string using a for loop, using the tuples in Eg_numb_functs
f1,f2=Eg_numb_functs[0]

f2_div=int(f2/f1)

stringer=f'{f1}*{f2_div}'

for item in Eg_numb_functs[1:]: #first item in stringer already
    eg,numb=item #eg,numb not all the same
    num_div=int(numb/eg)
    s=f'+{eg}*{num_div}'
    stringer += s
    
#Output
print(f"Which decomposes as follows:\n {P6_size}={stringer}")

Which decomposes as follows:
 5399325=114*125+121*65+126*237+138*112+158*248+162*247+165*132+169*117+173*132+178*122+182*106+183*132+187*24+192*134+194*123+208*144+209*22+216*234+217*126+218*61+221*125+227*22+229*307+230*250+232*120+233*119+236*122+241*30+242*102+247*61+248*53+250*114+251*59+260*184+268*106+269*64+271*186+273*118+274*56+278*52+279*41+281*136+284*112+289*67+293*133+294*37+300*133+305*67+313*123+324*123+325*132+326*101+329*181+338*120+343*59+345*113+356*19+381*121+390*44+397*107+419*20+420*46+421*23+422*39+429*64+433*35+436*128+437*23+455*64+477*53+496*48+502*16+509*38+518*12+520*107+524*20+531*36+538*76+539*62+553*36+555*66+566*144+578*61+605*62+607*66+613*11+640*28+659*30+759*48+765*21+781*108+802*119+814*42+881*42+938*8+974*135+977*45+983*135+1002*45+1026*126+1029*63+1033*63+1068*96+1121*108+1145*22+1178*39+1189*39+1209*44+1212*11+1224*66+1257*44+1312*11+1322*32+1377*36+1409*16+1421*10+1450*54+1511*20+1666*21+1753*24+2110*42+2193*95+2285*4+2290*54+3544*12+3673*42+3820

In [39]:
#Check output calculates 5399325 #Yes
s=114*125+121*65+126*237+138*112+158*248+162*247+165*132+169*117+173*132+178*122+182*106+183*132+187*24+192*134+194*123+208*144+209*22+216*234+217*126+218*61+221*125+227*22+229*307+230*250+232*120+233*119+236*122+241*30+242*102+247*61+248*53+250*114+251*59+260*184+268*106+269*64+271*186+273*118+274*56+278*52+279*41+281*136+284*112+289*67+293*133+294*37+300*133+305*67+313*123+324*123+325*132+326*101+329*181+338*120+343*59+345*113+356*19+381*121+390*44+397*107+419*20+420*46+421*23+422*39+429*64+433*35+436*128+437*23+455*64+477*53+496*48+502*16+509*38+518*12+520*107+524*20+531*36+538*76+539*62+553*36+555*66+566*144+578*61+605*62+607*66+613*11+640*28+659*30+759*48+765*21+781*108+802*119+814*42+881*42+938*8+974*135+977*45+983*135+1002*45+1026*126+1029*63+1033*63+1068*96+1121*108+1145*22+1178*39+1189*39+1209*44+1212*11+1224*66+1257*44+1312*11+1322*32+1377*36+1409*16+1421*10+1450*54+1511*20+1666*21+1753*24+2110*42+2193*95+2285*4+2290*54+3544*12+3673*42+3820*48+3983*18+6280*1+6474*6+6702*14+6960*16+7250*9+7580*4

5399325

# Leftovers (Please ignore)

In [ ]:
# def get_Pnp1_for_single_funct(Pn,n):
    
#     #inputs
#     # Pn:dataframe and n.
    
#     """
#     Returns a dataframe of extensions in Pn+1 for dataframe of functions in Pn.
#     """
#     """
#     #Example
#     n=3
#     power_set_str,power_set_str_np1=get_n_np1_powersets(n)
#     # P3

#     funct_list=[{'1': 0,'2': 0,"3":0,"12":0,"23":0,"13":0,"123":0}]
#     Pn=pd.DataFrame(funct_list)

#     #The following should be functions in P4 that are extendions of functions in funct_list
#     f_P3=get_Pnp1_for_single_funct(Pn,n)
#     print(f_P3)
#     """

#     #New data created
#     power_set_str,power_set_str_np1=get_n_np1_powersets(n)

#     #Load Pn as a list of dictionaries.
#     list_df_Pn=Pn.to_dict(orient='records')

#     #Create Pn+1 and hold as dataframe.
#     P_n1=eg_column_Build_Pn_1(list_df_Pn,power_set_str,n)
#     #As previous was done in E(g) parts, joint together
#     Pnp1=extend_sets_to_Pnp1(P_n1)
        
#     Pnp1=get_Eg_column_Pn(Pnp1)
    
#     columns=list(power_set_str_np1)+["Eg_Size"]
#     Pnp1 = Pnp1.reindex(columns=columns)
    
#     # inspect_Pn1(Pnp1)

#     return Pnp1

In [ ]:
# def check_all_funct_topspace(df,power_set_str):
#     length=[]
#     for i,g in enumerate(df.to_dict("records")): #list of dictionaries of functions
#         check_top=check_top_for_single_g(df,i,power_set_str)
#         if check_top == False:
#             print(f"Function {i} does NOT give a topolgical space: {check_top}")
#         else:
#             length.append(check_top)
            
#     if len(df.to_dict("records"))==len(length):
#         print("Every function produces a topological space" )
#     else:
#         print("something is not a topological space")

In [ ]:
# def is_finite_topological_space(X, T):
    
#     """
#         # Does this account for unions? 

#     # X = {'a', 'b', 'c'}
#     # T = [X, set(), {'a'}, { 'c'}, {'b'}]
#     # print(is_finite_topological_space(X, T)) # prints False

#     # This condition is not met because the union of {'a'} and {'c'} is {'a','c'} which is not in T.
#     """
    
#     if not all(isinstance(x, set) for x in T):
#         # T must be a collection of sets
#         print(x)
        
#         return False
#     if not all(x.issubset(X) for x in T):
#         # Each element of T must be a subset of X
#         print("here1")

#         return False
#     if not all(x.intersection(y) in T for x in T for y in T):
#         print("here2")

#         # The intersection of any two elements of T must be in T
#         return False
#     if not all(x.union(y) in T for x in T for y in T):
#         # The union of any two elements of T must be in T
#         print("here3")
#         return False
    
#     if not X in T:
#         print("here4")
#         print(x)

#         # X must be in T
#         return False
#     if not set() in T:
#         print("here5")

#         # The empty set must be in T
#         return False
#     return True

# def check_top_for_single_g(df,numb,power_set_str):
#     #Inputs:
#     # df=Pn
#     # numb=choice of function
#     # power_set_str for df
    
#     # Output:True or False
    
#     Eg=get_Eg_for_single_g(df,numb,power_set_str)
#     Top=list(set(x) for x in Eg)
#     Top.append(set())
    
#     X_Eg=set(power_set_str)
    
#     return is_finite_topological_space(X_Eg, Top)


# def check_df_duplicates(df1,df2,How,value):
#         """
#     Objective:
#     Input:
#     Returns:
#     """
#     #Compare df1,df2 if that common rows
    
#     # how is either "right" or "left"
#     # for left gives those in df1 and says True or false if also in df2
    
#     #     example
#     #     df1 = pd.DataFrame({'team' : ['A', 'B', 'C', 'D', 'E'], 
#     #                     'points' : [12, 15, 22, 29, 24]}) 
#     #  #create second DataFrame
#     # df2 = pd.DataFrame({'team' : ['A', 'D', 'F', 'G', 'H'],
#     #                     'points' : [12, 29, 15, 19, 10]})
    
#     #merge two dataFrames and add indicator column
#     all_df = pd.merge(df1, df2, how=How, indicator='exists')

#     #add column to show if each row in first DataFrame exists in second
#     all_df['exists'] = np.where(all_df.exists == 'both', True, False)

#     #view updated DataFrame
#     # print (all_df)

#     m=all_df.loc[all_df['exists'] == True]
    
#     #Number of terms common
#     num_items = m.loc[m['exists'] == value].shape[0]
#     print(f"Number of those in both: {num_items} \n")

#     return m    


## Examples for understanding code

Are primitive closed sets a basis for the topology? (Maybe not relevant)

In [ ]:

Asides:
- Is Eg a matriod? No does not satisfy downward closure. In our sepcific example f(12)=1,f(123)=1 else 0 Take {1} in the closure of 1 i.e {1,13}, {1} is not a closed set.
- Are primite closed sets a basis for the topology? Is there a way to construct $\mathbb{E}(g)$ using primitive (using minimal basis).

In [ ]:
# dict_PC,set_PC=get_primitive_for_single_g(df,numb)

# # print(dict_PC) #Remembers what the terms of powerset is
# set_PC.append(frozenset()) # add empty set: this is the set of primitive closed sets
# # for i in set_PC:
# #     print(set(i))
    
# X_Eg= {'13', '23', '123', '12', '2', '3', '1'} #total space ie powerset
# Prim_Top= list(set(x) for x in set_PC) # primitive set as a topology? is it a basis?

# def is_basis(X, T, B):
#     if not all(isinstance(x, set) for x in B):
#         # B must be a collection of sets
#         return False
#     if not all(x.issubset(X) for x in B):
#         # Each element of B must be a subset of X
#         return False
    
    
#     if not all(x.intersection(y) == set() for x in B for y in B if x != y):
#         # The intersection of any two distinct elements of B must be empty
#         return False
    
#     if not all(x.union(y) in T for x in B for y in B):
#         # The union of any two elements of T must be in T
#         return False
    
#     if not all(x.issubset(y) or y.issubset(x) for x in B for y in T):
#         # Each element of B must be a subset of some element of T or conversely
#         return False
#     return True

# # Example usage
# X = {'a', 'b', 'c'}
# T = [X, set(), {'a', 'b'}, {'b', 'c'}, {'b'}]
# B = [{'a', 'b'}, {'b', 'c'}]
# print(is_basis(X, T, B)) # prints True


In [ ]:
#Get all functions in P3 stored a data frame

#Construct P3
pre_functs=[["12","123"],["13","123"],["23","123"],["12","13","123"],["12","23","123"],["13","23","123"],["12","13","23","123"],[],["123"]]
ds=[]
for vals in pre_functs:
    builder1=[(x,1) for x in vals]
    builder0=[(x,0) for x in power3 if x not in vals]
    f_dict={**dict(builder1),**dict(builder0)}
    ds.append(f_dict)
big={'123': 2, '1': 0, '2': 0, '3': 0, '12': 1, '13': 1, '23': 1} #remaining case, with 2 value
dp=ds+[big,]

#Build dataframe
P3 = pd.DataFrame(dp)

#order columns of P3
power3=get_powerset_str(3) #powerset on 3 elements #for 
column_order=power3 #as strings
P3 = P3.reindex(columns=column_order)

print(P3)

In [ ]:
## Checking if the calculations for P5, are the same as our old calculations for P5 in xlsx.

# #Previous data P5 in correct format
# dataframe2 = pd.read_excel('P4_P5_xlsx\P_5.xlsx')

# #Make sure columns are in the same order as P5

# #Reordering columns
# column_order=[int(x) for x in power_set_str_np1]
# dataframe2 = dataframe2.reindex(columns=column_order)

# #Making sure columns are the same, in string format
# dataframe2=string_cols(dataframe2)

# # print(dataframe2.head)
# print(dataframe2.shape) #(10334, 31)

# # n=4
# # power_set_str,alt_power_set_str_np1=get_n_np1_powersets(n)
# # check_functs_MSA(dataframe2,power_set_str_np1).shape #(0,0) i.e all are msa

# Comparision of P5 and dataframe 2. We see they are the same.

# compar=check_df_duplicates(P5,dataframe2,"left",True) #True:10334 , False:0

In [ ]:
# # The calculations for P4, are the same as our old calculations for P4 in xlsx.

# dataframe1 = pd.read_excel('P4_P5_xlsx\P_4.xlsx')
# #Making sure columns are the same, in string format and order as P4 above
# dataframe1=string_cols(dataframe1)

# # #Comparision of P4 and xlsx version
# compar=check_df_duplicates(P4,dataframe1,"left",True)

In [ ]:
# n=3
# # Create a set with 3 elements
# # s = {1, 2, 3}
# s=set(range(1,n+1))

# # Generate all possible subsets of the set
# power_set = [set(x)  for r in range(len(s) + 1) for x in itertools.combinations(s, r)]

# # Print the power set
# print(power_set)

# power_set_str=[set_to_string(x) for x in power_set]
# power_set_str=[set_to_string(x) for x in power_set if len(x)>0]
# power_set_str

# #Get a function in P3

# val1_loc=["12","123"] #where it take 1
# val0_loc= [x for x in power_set_str if x not in val1_loc] #where it takes 0
# # print(val0_loc)

# builder1=[(x,1) for x in val1_loc]
# builder0=[(x,0) for x in val0_loc]
# builder0
# dict(builder0)
# dict(builder1)
# f_dict={**dict(builder1),**dict(builder0)}
# f_dict

# A,B="1","123"
# print(f"Is {A} subset of {B}, f minimal? : {fmin(A,B,f_dict)}")
# print(f"Is {A} subset of {B}, f maximal? : {fmax(A,B,f_dict)}")

#Testing for obtaining primitive sets:
# #Example
# for x in power_set_str:
#     print(x,get_barj({x},f_dict))

#As all the closed sets, checked by hand:
# 3 T term {'13', '123', '1', '3', '23', '2'}
# 1 {'1', '13'}
# 2 {'23', '2'}
# 12 {'1', '12', '123', '13', '2', '23'} 
# 13 {'13'} # should be just 13
# 23 {'23'} # should be 23
# 123 {'13', '123', '1', '23', '2'} # should not have 3 or 12

# Store primitive closed sets 

# PC=[(x,get_barj({x},f_dict)) for x in power_set_str]
# dict_PC=dict(PC)
# set_PC=[sorted_frozenset(get_barj({x},f_dict)) for x in power_set_str]
# # set(set_PC)
# Eg={sorted_frozenset(power_set_str)}.union(set_PC)

# Using main functions:
# Eg=list(get_Eg(dict_PC,power_set_str,set_PC))
# # print(len(x)) # I get like 13 so this is an issue.
# Eg
# Eg.append("empty")
# Eg


#Rows are not in the same order:
# # print(P4.equals(dataframe1))

#Checking if duplicates, #https://stackoverflow.com/questions/48647534/find-difference-between-two-data-frames?noredirect=1&lq=1
# df = P4.merge(dataframe1, how = 'inner' ,indicator=False)
""" 
# first dataframe
df1 = pd.DataFrame({
    'Age': ['20', '14', '56', '28', '10'],
    'Weight': [59, 29, 73, 56, 48]})
display(df1)

# second dataframe
df2 = pd.DataFrame({
    'Age': ['16', '20', '24', '40', '22'],
    'Weight': [55, 59, 73, 85, 56]})
display(df2)

#Common rows:
df = df1.merge(df2, how = 'inner' ,indicator=False)
print(df)
"""

"The above method only works for those data frames that don't already have duplicates themselves. For example:"
no_dups=pd.concat([P4,dataframe1]).drop_duplicates(keep=False)
no_dups.head()


## E(g) is a topolgical space <a name="s4"></a>

### Example f in P3 form a Topological space <a name="s41"></a>

The set $\mathbb{E}(g)$ is the topology assoicated to the function $g$ and can be used to construct the extensions of $g$.

We now have:
- A function/method to get $\mathbb{E}(g)$.
- A function to check if $\mathbb{E}(g)$ is a finite topology.
- Check if all function in P3,P4,P5 generate a topology: Answer P3,P4 yes, examples of P5 yes

What we could do next:
- A way to determine the size of $\mathbb{E}(g)$ (using basis).
- Are all Top(g) for $g \in P_n$ homeomorphic? (topologies are different sizes).

### Specific Example from P3 is topological space

In [26]:
power_set_str=get_powerset_str(3)

df=P3
numb=4
#Get data
Eg=get_Eg_for_single_g(df,numb,power_set_str)
# print(power_set_str)
print(P3.iloc[numb].to_dict(),"\n")

for term in Eg:
    print(set(term))
    
print("\n  All closed sets")


{'1': 0, '2': 0, '3': 0, '12': 0, '13': 0, '23': 0, '123': 0} 

{'12', '123', '23'}
{'12', '1', '123', '13'}
{'123', '3', '23', '13'}
{'3', '2', '23', '12', '1', '123', '13'}
{'123'}
{'3', '23', '12', '123', '13'}
{'2', '23', '12', '1', '123', '13'}
{'12', '123', '23', '13'}
{'12', '123', '13'}
{'23', '12', '1', '123', '13'}
{'12', '23', '123', '2'}
{'12', '123'}
{'123', '23'}
{'3', '23', '2', '12', '123', '13'}
{'123', '13'}
{'23', '2', '12', '123', '13'}
{'123', '23', '13'}
{'3', '23', '12', '1', '123', '13'}
set()

  All closed sets


Check if Eg is a finite topological space on the powerset for an example g in P3

In [ ]:
X_Eg= {'13', '23', '123', '12', '2', '3', '1'} #total space ie powerset
Top=list(set(x) for x in Eg) #topology

print(is_finite_topological_space(X_Eg, Top))

### Consider Top(g) for two different g,g' in P3.

In [ ]:
#Setup data
power_set_str=get_powerset_str(3)
df=P3
X_Eg= set(power_set_str) #total space ie powerset

#Inputs
numb1=0
numb2=1

#Top space 1
Eg1=get_Eg_for_single_g(df,numb,power_set_str)
Top1=list(set(x) for x in Eg1) #topology
print(is_finite_topological_space(X_Eg, Top1))
#------------------------------
#Top space 2
Eg2=get_Eg_for_single_g(df,numb2,power_set_str)
Top2=list(set(x) for x in Eg2) #topology
print(is_finite_topological_space(X_Eg, Top2))
#------------------------------



### Check if all function in P3,P4,P5 generate a topological space: Yes <a name="s42"></a>

### P3

In [ ]:
#inputs
df=P3
power_set_str=get_powerset_str(3)
check_all_funct_topspace(df,power_set_str)

### P4

In [ ]:
#inputs
df=P4
power_set_str=get_powerset_str(4)
check_all_funct_topspace(df,power_set_str) #False

### P5 (where Eg for a function allow for calc of part of P6)

In [ ]:
#Take long: 

#inputs
df=P5
power_set_str=get_powerset_str(5)
check_all_funct_topspace(df,power_set_str)

In [ ]:
#Works for some choosen example.

df=P5
power_set_str=get_powerset_str(5)
numb=0 # function indexed at 0

Eg=get_Eg_for_single_g(df,numb,power_set_str) # set of closed sets
X_Eg= set(power_set_str) #total space ie powerset
Top=list(set(x) for x in Eg) #topology from closed sets Eg #Size: 1178

print(len(Top))

#Check whether topological space:
print(is_finite_topological_space(X_Eg, Top)) #The total powerset on 4 elements is not included.

#Result:
#numb=0
#1178 #Top size
#True